# Seq2Seq - Encoder/Decoder networks
In this exercise we'll have a deeper look into the ability to use multiple RNN's to infer and generate sequences of data.
Specifically we will implement a Encoder-Decoder RNN based for a simple sequence to sequence translation task.
This type of models have shown impressive performance in Neural Machine Translation and Image Caption generation. 

In the encoder-decoder structure one RNN (blue) encodes the input into a hidden representation, and a second RNN (red) uses this representation to predict the target values.
An essential step is deciding how the encoder and decoder should communicate.
In the simplest approach you use the last hidden state of the encoder to initialize the decoder.
This is what we will do in this notebook, as shown here:

![](./images/enc-dec.png)

In this exercise we will translate from the words of number (e.g. 'nine') to the actual number (e.g. '9').
The input for the Encoder RNN consists of words defining the number, whilst the output of such an encoding serves as input for the Decoder RNN that aims to generate generate a number. 
Our dataset is generated and consists of numbers and an End-of-Sentence (EOS) character ('#'). The data we want to generate should be like follows:

```
Examples: 
prediction  |  input
991136#00 	 nine nine one one three six
81771#000 	 eight one seven seven one
3519614#0 	 three five one nine six one four
26656#000 	 two six six five six
60344#000 	 six zero three four four
162885#00 	 one six two eight eight five
78612625# 	 seven eight six one two six two five
9464710#0 	 nine four six four seven one zero
191306#00 	 one nine one three zero six
10160378# 	 one zero one zero six three seven eight
```

Let us define the space of characters and numbers to be learned with the networks:

```
Number of valid characters: 27
'0'=0,	'1'=1,	'2'=2,	'3'=3,	'4'=4,	'5'=5,	'6'=6,	'7'=7,	'8'=8,	'9'=9,	'#'=10,	' '=11,	'e'=12,	'g'=13,	'f'=14,	'i'=15,	'h'=16,	'o'=17,	'n'=18,	's'=19,	'r'=20,	'u'=21,	't'=22,	'w'=23,	'v'=24,	'x'=25,	'z'=26,	
Stop/start character = #
```

All represented characters and numbers as characters, gets mapped to an integer from 0-26. Our total space of valid characters consists of 27.

In [160]:
from data_generator import generate
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from data_generator import generate

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device in use:", device)

NUM_INPUTS = 27 #No. of possible characters
NUM_OUTPUTS = 11  # (0-9 + '#')

### Hyperparameters and general configs
MAX_SEQ_LEN = 8
MIN_SEQ_LEN = 5
BATCH_SIZE = 8
TRAINING_SIZE = 8000
LEARNING_RATE = 0.003

# Hidden size of enc and dec need to be equal if last hidden of encoder becomes init hidden of decoder
# Otherwise we would need e.g. a linear layer to map to a space with the correct dimension
NUM_UNITS_ENC = NUM_UNITS_DEC = 96
TEST_SIZE = 200
EPOCHS = 15
TEACHER_FORCING = False

assert TRAINING_SIZE % BATCH_SIZE == 0

Device in use: cuda


For this exercise we won´t worry about data generation, but utilise a built function for this purpose. The function generates random data constained by the 27 characters described above.

The encoder takes as input the embedded text strings generated from the *generate* function as given here above ie. 'nine' would become [18 15 18 12].
Sequeneces are generated at random given settings of minima and maxima length, constrained by the dimensions of the two RNN´s architecture.
We may visualise a subset of the data generated by running the command below

In [151]:
!python3 ./data_generator.py

Generated batch length 3 from 3 iterations
input types: int32 int32 int32 int32 int32
Number of valid characters: 27
'0'=0,	'1'=1,	'2'=2,	'3'=3,	'4'=4,	'5'=5,	'6'=6,	'7'=7,	'8'=8,	'9'=9,	'#'=10,	'r'=11,	' '=12,	'n'=13,	'v'=14,	'i'=15,	'e'=16,	'x'=17,	'z'=18,	'u'=19,	'h'=20,	'w'=21,	's'=22,	'g'=23,	't'=24,	'f'=25,	'o'=26,	
Stop/start character = #

SAMPLE 0
TEXT INPUTS:			 two seven three
ENCODED INPUTS:			 [24 21 26 12 22 16 14 16 13 12 24 20 11 16 16  0  0  0]
INPUTS SEQUENCE LENGTH:	 15
TEXT TARGETS INPUT:		 #273
TEXT TARGETS OUTPUT:	 273#
ENCODED TARGETS INPUT:	 [10  2  7  3  0]
ENCODED TARGETS OUTPUT:	 [ 2  7  3 10  0]
TARGETS SEQUENCE LENGTH: 4
TARGETS MASK:			 [1. 1. 1. 1. 0.]

SAMPLE 1
TEXT INPUTS:			 four nine two five
ENCODED INPUTS:			 [25 26 19 11 12 13 15 13 16 12 24 21 26 12 25 15 14 16]
INPUTS SEQUENCE LENGTH:	 18
TEXT TARGETS INPUT:		 #4925
TEXT TARGETS OUTPUT:	 4925#
ENCODED TARGETS INPUT:	 [10  4  9  2  5]
ENCODED TARGETS OUTPUT:	 [ 4  9  2  5 10]
TARGETS SEQUENCE LENG

## Let's define the two RNN's



In [152]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, self.hidden_size)
        rnn = nn.LSTM
        self.rnn = rnn(self.hidden_size, self.hidden_size, batch_first=True)

    def forward(self, inputs, hidden, cn):
        # Input shape [batch, seq_in_len]z
        inputs = inputs.long()

        # Embedded shape [batch, seq_in_len, embed]
        embedded = self.embedding(inputs)
        
        # Output shape [batch, seq_in_len, embed]
        # Hidden shape [1, batch, embed], last hidden state of the GRU cell
        # We will feed this last hidden state into the decoder
        output, (hidden, cn) = self.rnn(embedded, (hidden, cn))
        return output, hidden, cn

    def init_hidden(self, batch_size):
        init = torch.zeros(1, batch_size, self.hidden_size, device=device)
        return init

    def init_cn(self, batch_size):
        init = torch.zeros(1, batch_size, self.hidden_size, device=device)
        return init


In [153]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        rnn = nn.LSTM
        self.rnn = rnn(self.hidden_size, self.hidden_size, batch_first=True)

    def forward(self, inputs, hidden, cn, output_len, teacher_forcing=False):
        # Input shape: [batch, output_len]
        # Hidden shape: [seq_len=1, batch_size, hidden_dim] (the last hidden state of the encoder)

        if teacher_forcing:
            dec_input = inputs
            embed = self.embedding(dec_input)   # shape [batch, output_len, hidden_dim]
            out, (hidden, cn) = self.rnn(embed, (hidden, cn))
            out = self.out(out)  # linear layer, out has now shape [batch, output_len, output_size]
            output = F.log_softmax(out, -1)
        else:
            # Take the EOS character only, for the whole batch, and unsqueeze so shape is [batch, 1]
            # This is the first input, then we will use as input the GRU output at the previous time step
            dec_input = inputs[:, 0].unsqueeze(1)

            output = []
            for i in range(output_len):
                out, (hidden, cn) = self.rnn(self.embedding(dec_input), (hidden, cn))
                out = self.out(out)  # linear layer, out has now shape [batch, 1, output_size]
                out = F.log_softmax(out, -1)
                output.append(out.squeeze(1))
                out_symbol = torch.argmax(out, dim=2)   # shape [batch, 1]
                dec_input = out_symbol   # feed the decoded symbol back into the recurrent unit at next step

            output = torch.stack(output).permute(1, 0, 2)  # [batch_size x seq_len x output_size]

        return output

The learned representation from the *Encoder* gets propagated to the *Decoder* as the final hidden layer in the *Encoder* network is set as initialisation for the *Decoder*'s first hidden layer.

In [154]:
def forward_pass(encoder, decoder, x, t, t_in, criterion, max_t_len, teacher_forcing):
    """
    Executes a forward pass through the whole model.

    :param encoder:
    :param decoder:
    :param x: input to the encoder, shape [batch, seq_in_len]
    :param t: target output predictions for decoder, shape [batch, seq_t_len]
    :param criterion: loss function
    :param max_t_len: maximum target length

    :return: output (after log-softmax), loss, accuracy (per-symbol)
    """
    # Run encoder and get last hidden state (and output)
    batch_size = x.size(0)
    enc_h = encoder.init_hidden(batch_size)
    enc_cn = encoder.init_cn(batch_size)
    enc_out, enc_h, enc_cn = encoder(x, enc_h, enc_cn)

    dec_h = enc_h  # Init hidden state of decoder as hidden state of encoder
    dec_cn = enc_cn
    dec_input = t_in
    out = decoder(dec_input, dec_h, dec_cn, max_t_len, teacher_forcing)
    out = out.permute(0, 2, 1)
    # Shape: [batch_size x num_classes x out_sequence_len], with second dim containing log probabilities

    loss = criterion(out, t)
    pred = get_pred(log_probs=out)
    accuracy = (pred == t).type(torch.FloatTensor).mean()
    return out, loss, accuracy


In [155]:
def train(encoder, decoder, inputs, targets, targets_in, criterion, enc_optimizer, dec_optimizer, epoch, max_t_len):
    encoder.train()
    decoder.train()
    for batch_idx, (x, t, t_in) in enumerate(zip(inputs, targets, targets_in)):
        encoder.zero_grad()
        decoder.zero_grad()
        
        out, loss, accuracy = forward_pass(encoder, decoder, 
                                           x.to(device), 
                                           t.to(device).long(), 
                                           t_in.to(device).long(), 
                                           criterion, 
                                           max_t_len, 
                                           TEACHER_FORCING)
        loss.backward()
        
        enc_optimizer.step()
        dec_optimizer.step()
        
        
        if batch_idx % 200 == 0:
            print('Epoch {} [{}/{} ({:.0f}%)]\tTraining loss: {:.4f} \tTraining accuracy: {:.1f}%'.format(
                epoch, batch_idx * len(x), TRAINING_SIZE,
                100. * batch_idx * len(x) / TRAINING_SIZE, loss.item(),
                100. * accuracy.item()))


In [156]:
def test(encoder, decoder, inputs, targets, targets_in, criterion, max_t_len):
    encoder.eval()
    decoder.eval()
    with torch.no_grad():
        inputs = inputs.to(device)
        targets = targets.long().to(device)
        targets_in = targets_in.long().to(device)
        out, loss, accuracy = forward_pass(encoder, decoder, inputs, targets, targets_in, criterion, max_t_len,
                                           teacher_forcing=TEACHER_FORCING)
    return out, loss, accuracy

In [157]:
def numbers_to_text(seq):
    return "".join([str(to_np(i)) if to_np(i) != 10 else '#' for i in seq])

def to_np(x):
    return x.cpu().numpy()

def get_pred(log_probs):
    """
    Get class prediction (digit prediction) from the net's output (the log_probs)
    :param log_probs: Tensor of shape [batch_size x n_classes x sequence_len]
    :return:
    """
    return torch.argmax(log_probs, dim=1)

In [161]:
encoder = EncoderRNN(NUM_INPUTS, NUM_UNITS_ENC).to(device)
decoder = DecoderRNN(NUM_UNITS_DEC, NUM_OUTPUTS).to(device)
enc_optimizer = optim.RMSprop(encoder.parameters(), lr=LEARNING_RATE)
dec_optimizer = optim.RMSprop(decoder.parameters(), lr=LEARNING_RATE)
criterion = nn.NLLLoss().to(device)

# Get training seta
inputs, _, targets_in, targets, targets_seqlen, _, _, _, text_targ = generate(TRAINING_SIZE, min_len=MIN_SEQ_LEN, max_len=MAX_SEQ_LEN)
max_target_len = max(targets_seqlen)
inputs = torch.tensor(inputs)
targets = torch.tensor(targets)
targets_in = torch.tensor(targets_in)
unique_text_targets = set(text_targ)

# Get validation set
val_inputs, _, val_targets_in, val_targets, val_targets_seqlen, _, val_text_in, _, val_text_targ = \
    generate(TEST_SIZE, min_len=MIN_SEQ_LEN, max_len=MAX_SEQ_LEN, invalid_set=unique_text_targets)
val_inputs = torch.tensor(val_inputs)
val_targets = torch.tensor(val_targets)
val_targets_in = torch.tensor(val_targets_in)
max_val_target_len = max(val_targets_seqlen)
test(encoder, decoder, val_inputs, val_targets, val_targets_in, criterion, max_val_target_len)

# Split training set in batches
inputs = [inputs[i * BATCH_SIZE: (i + 1) * BATCH_SIZE] for i in range(TRAINING_SIZE // BATCH_SIZE)]
targets = [targets[i * BATCH_SIZE: (i + 1) * BATCH_SIZE] for i in range(TRAINING_SIZE // BATCH_SIZE)]
targets_in = [targets_in[i * BATCH_SIZE: (i + 1) * BATCH_SIZE] for i in range(TRAINING_SIZE // BATCH_SIZE)]

# Quick and dirty - just loop over training set without reshuffling
for epoch in range(1, EPOCHS + 1):
    train(encoder, decoder, inputs, targets, targets_in, criterion, enc_optimizer, dec_optimizer, epoch, max_target_len)
    _, loss, accuracy = test(encoder, decoder, val_inputs, val_targets, val_targets_in, criterion, max_val_target_len)
    print('\nTest set: Average loss: {:.4f} \tAccuracy: {:.3f}%\n'.format(loss, accuracy.item()*100.))

    # Show examples
    print("Examples: prediction | input")
    out, _, _ = test(encoder, decoder, val_inputs[:10], val_targets[:10], val_targets_in[:10], criterion, max_val_target_len)
    pred = get_pred(out)
    pred_text = [numbers_to_text(sample) for sample in pred]
    for i in range(10):
        print(pred_text[i], "\t", val_text_in[i])
    print()

Generated batch length 8000 from 8000 iterations
Generated batch length 200 from 201 iterations
Epoch 1 [0/8000 (0%)]	Training loss: 2.3690 	Training accuracy: 18.1%
Epoch 1 [1600/8000 (20%)]	Training loss: 1.8130 	Training accuracy: 33.3%
Epoch 1 [3200/8000 (40%)]	Training loss: 1.9307 	Training accuracy: 29.2%
Epoch 1 [4800/8000 (60%)]	Training loss: 1.7119 	Training accuracy: 38.9%
Epoch 1 [6400/8000 (80%)]	Training loss: 1.3157 	Training accuracy: 51.4%

Test set: Average loss: 1.4343 	Accuracy: 45.944%

Examples: prediction | input
157777#00 	 one seven five five zero six
5333333#0 	 five seven five six three nine eight
388222#00 	 two five two three one two
3393999## 	 three zero nine four six seven nine
5777777## 	 four two seven seven eight one four seven
577777#00 	 five six eight seven one seven
8626222#0 	 eight eight seven four four six two
602222#00 	 seven zero eight six four two
1020222#0 	 nine three zero four eight two three
3333888#0 	 two eight four four three seven 

Epoch 10 [1600/8000 (20%)]	Training loss: 0.1794 	Training accuracy: 93.1%
Epoch 10 [3200/8000 (40%)]	Training loss: 0.3685 	Training accuracy: 88.9%
Epoch 10 [4800/8000 (60%)]	Training loss: 0.1732 	Training accuracy: 93.1%
Epoch 10 [6400/8000 (80%)]	Training loss: 0.1654 	Training accuracy: 93.1%

Test set: Average loss: 0.3821 	Accuracy: 88.056%

Examples: prediction | input
175506#00 	 one seven five five zero six
5756798#0 	 five seven five six three nine eight
252112#00 	 two five two three one two
3094679#0 	 three zero nine four six seven nine
32771147# 	 four two seven seven eight one four seven
568177#00 	 five six eight seven one seven
88746622# 	 eight eight seven four four six two
708642#00 	 seven zero eight six four two
9304823#0 	 nine three zero four eight two three
2843378#0 	 two eight four four three seven eight

Epoch 11 [0/8000 (0%)]	Training loss: 0.2416 	Training accuracy: 88.9%
Epoch 11 [1600/8000 (20%)]	Training loss: 0.1785 	Training accuracy: 93.1%
Epoch 11 

# Exercise:

1. Implement missing code for the network in the *train* function. 
2. These networks implement the GRU-gates. Implement an alternative control utilising a memory mechanism (Hint: LSTM). What do you experience? 
3. (optional) There are some parameters in the model that may be optimized further, what could they be? Achieve >90% validation accuracy.

**Answers:**

1. Code can be seen in the above cells. However, I got a validation acc. of ~75% instead, but it varied a lot from training to training. 
2. Code can be seen in the above cells. The validation acc. seemed to be about the same, but appeared more stable.
3. I increased the number of hidden units to 96. This gave a validation acc. of ~95%.

### Exercise from book
I chose [this](http://neuralnetworksanddeeplearning.com/chap3.html#exercise_332838) exercise:
Construct an example showing explicitly that in a network with a sigmoid output layer, the output activations $a^L_j$ won't always sum to 1. 

**Answer:**
The sigmoid function is
$$\sigma(z) = \frac{1}{1 + e^{-z}}. $$
Let's assume we have two values $z_1 = 0$ and $z_2 = 1$. Then, the sum of the output activations are:
$$ \sigma(1) + \sigma(0) \approx 0.73 + 0.5 = 1.23 \neq 1, $$
which concludes the proof.
